In [ ]:
{
  "event_id": "evt_550e8400-e29b",
  "aggregate_id": "acc_1001",       // ID счета отправителя
  "aggregate_type": "BankAccount",
  "event_type": "MoneyTransferred",
  "version": 15,                    // Версия агрегата для оптимистической блокировки
  "timestamp": "2024-12-27T10:00:00Z",
  "payload": {
    "from_account": "acc_1001",
    "to_account": "acc_2005",
    "amount": 5000.00,
    "currency": "RUB",
    "transaction_ref": "tx_987654321"
  },
  "metadata": {
    "user_id": "user_777",
    "ip_address": "192.168.1.1",
    "device": "iPhone 14"
  }
}

In [ ]:
---
config:
  layout: elk
---
graph TD
    Client["Мобильное приложение"]

    subgraph "Command Side (Запись)"
        API["Command Service"]
        EventStore[("MongoDB: Event Store")]
    end

    subgraph "Event Bus"
        Kafka{"Apache Kafka"}
    end

    subgraph "Query Side (Чтение)"
        Projector["Balance Projector"]
        Redis[("Redis: Current State")]
        QueryAPI["Query Service"]
    end

    subgraph "Analytics"
        ETL["Analytics Worker"]
        DWH[("ClickHouse: OLAP")]
    end
    Client -->|"1. Перевод (POST)"| API
    API -->|"2. Сохранить событие"| EventStore
    API -->|"3. Опубликовать событие"| Kafka

    Kafka -->|"4. Подписка"| Projector
    Projector -->|"5. Обновить баланс"| Redis

    Kafka -->|"4. Подписка"| ETL
    ETL -->|"5. Вставка"| DWH

    Client -.->|"6. Узнать баланс (GET)"| QueryAPI
    QueryAPI -.->|"7. Чтение"| Redis
    style EventStore fill:#d4edda,stroke:#28a745
    style Redis fill:#ffeeba,stroke:#ffc107
    style DWH fill:#cce5ff,stroke:#004085
    style Kafka fill:#e2e3e5,stroke:#333